In [8]:
import numpy as np
import pandas as pd
import neuralnetworksA2 as nn
import imp
from PIL import Image
import os
import numpy as np
import scipy.misc as smp
# X = np.arange(10*2).reshape((10, 2))
# T = X[:, 0:1] * 0.1


# def partition(X, T, rate, shuffle):
#     nRows = len(X)
#     rows = np.arange(nRows)
#     if(shuffle):
#         np.random.shuffle(rows)
#     nTrain = int(nRows * rate)
#     trainRows = rows[:nTrain]
#     testRows = rows[nTrain:]
#     Xtrain, Ttrain = X[len(trainRows):], T[len(trainRows):]
#     Xtest, Ttest = X[len(testRows):], T[len(testRows):]
#     return Xtrain, Ttrain, Xtest, Ttest

def partition(X, T, fraction, shuffle):
    X = np.array(X)
    T = np.array(T)
    
    if shuffle:
        np.random.shuffle(X)
        np.random.shuffle(T)
    
    xTrain = X[:int(fraction * X.shape[0])]
    tTrain = T[:int(fraction * T.shape[0])]
    xTest  = X[int(fraction * X.shape[0]):]
    tTest  = T[int(fraction * T.shape[0]):]
    
    return xTrain, tTrain, xTest, tTest

In [2]:
directory_str = './flags-normal/train-data/'
directory     = os.fsencode(directory_str)

array_of_pictures = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"):
        img = Image.open(directory_str+filename)
        pixel_index = img.load()
        height, width = img.size
        data = np.array(img.getdata())
        picture = []
        for i in range(height):
            row = []
            for j in range(width):
                row.append(pixel_index[i, j])
            picture.append(row)
        np_picture = np.array(picture)
    if  len(np_picture.shape) == 3 and np_picture.shape[2] == 3 :
        array_of_pictures.append(picture)
        



In [3]:
# DEBUGGING ONLY
array_of_pictures = [array_of_pictures[0]]

In [4]:
picture = array_of_pictures[0]
print(len(array_of_pictures))
print(np.array(picture).shape)
# Picture is a list [ of rows of list[ where each pixel in a row is a list of 3 values [x x x]]]
# picture[0].append((0,0,0))
# print(np.array(picture).shape)


#Standardizing pictures so they all have the same dimension
# for picture in array_of_pictures:
#     for row in picture:
#         while len(row) < 367:
#             row.append((0,0,0))
# #     print(np.array(picture).shape)

stream_of_pictures = [row for picture in array_of_pictures for row in picture]

1
(550, 275, 3)


In [5]:
# This code verifies dimensions of stream_of_pictures

print(len(stream_of_pictures))
print(np.array(stream_of_pictures).shape)
for row in stream_of_pictures:
    print(len(row))
    print(len(row[0]))

550
(550, 275, 3)
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275
3
275


In [6]:
imp.reload(nn)


# Split up picture
X = np.array([row[int(0.2 * len(row)):] for row in stream_of_pictures])
# print(len(X))
T = np.array([row[0:int(0.2 * len(row))] for row in stream_of_pictures])

print("Shapes before reshape")
print(X.shape)
print(T.shape)
# print(len(T))
X_reshaped = X.reshape((X.shape[0], -1))
T_reshaped= T.reshape((T.shape[0], -1))

print("\nShapes after reshape")
print(X_reshaped.shape)
print(T_reshaped.shape)     

Xtrain, Ttrain, Xtest, Ttest = partition(X_reshaped, T_reshaped, 0.8, shuffle=False)
Xtrain_std, Ttrain_std, Xtest_std, Ttest_std = partition(X, T, 0.8, shuffle=False)


print("\nStandard shapes")
print("XTrain_std: "+str(Xtrain_std.shape))
print("XTest_std: "+str(Xtest_std.shape))
print("Ttrain_std: "+str(Ttrain_std.shape))
print("TTest_std: "+str(Ttest_std.shape)+"\n")

print("\nShapes ")
print("XTrain: "+str(Xtrain.shape))
print("XTest: "+str(Xtest.shape))
print("Ttrain: "+str(Ttrain.shape))
print("TTest: "+str(Ttest.shape)+"\n")
netc = nn.NeuralNetwork(X_reshaped.shape[1], [1], T_reshaped.shape[1])
netc.train(X_reshaped, T_reshaped, 20)
# print(netc)
# print('T, Predicted')
# print(np.hstack((Ttest, netc.use(Xtest))))

print(netc.use(X_reshaped).shape)
predicted_values = netc.use(X_reshaped).reshape(T.shape)
print(predicted_values.shape)



Shapes before reshape
(550, 220, 3)
(550, 55, 3)

Shapes after reshape
(550, 660)
(550, 165)

Standard shapes
XTrain_std: (440, 220, 3)
XTest_std: (110, 220, 3)
Ttrain_std: (440, 55, 3)
TTest_std: (110, 55, 3)


Shapes 
XTrain: (440, 660)
XTest: (110, 660)
Ttrain: (440, 165)
TTest: (110, 165)

(550, 165)
(550, 55, 3)


In [9]:
print(netc.use(X_reshaped).shape)
predicted_values = netc.use(X_reshaped).reshape(T.shape)
print(predicted_values.shape)
print(X.shape)

predict_pic = np.append(predicted_values, X, axis = 1)
print(predict_pic.shape)

# data = np.zeros((len(picture[0]),len(picture[0][0]),3), dtype=np.uint8 )

img = smp.toimage(predict_pic)       # Create a PIL image
img.show()


(550, 165)
(550, 55, 3)
(550, 220, 3)
(550, 275, 3)


In [ ]:
import numpy as np
import scipy.misc as smp
imp.reload(nn)

picture1 = picture
# Create a 1024x1024x3 array of 8 bit unsigned integers
data = np.zeros( (len(picture1[0]),len(picture1[0][0]),3), dtype=np.uint8 )

for row in range(0,len(picture1[0])):
    for col in range(0,len(picture1[0][0])):
        data[row][col] = picture1[0][row][col]

img = smp.toimage(data)       # Create a PIL image
img.show()